In [1]:
# This snippet of code properly adds the working source root path to python's path
# so you no longer have to install spykshrk through setuptools
import sys, os 
root_depth = 2
notebook_dir = globals()['_dh'][0]
root_path = os.path.abspath(os.path.join(notebook_dir, '../'*root_depth))
# Add to python's path
try:
    while True:
        sys.path.remove(root_path)
except ValueError:
    # no more root paths
    pass
sys.path.append(root_path)
# Alternatively set root path as current working directory
#os.chdir(root_path)

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import functools
import itertools
import holoviews as hv
from holoviews import dim, opts
import enum
import IPython
import warnings

from IPython.utils.text import columnize
from IPython.lib.pretty import pprint, pretty
from IPython.display import display_html

# Spykshrk modules for data analysis
from spykshrk.franklab.data_containers import FlatLinearPosition, SpikeFeatures, \
        EncodeSettings, pos_col_format, SpikeObservation
from spykshrk.franklab.generator.place.data_generator import UnitNormalGenerator, TetrodeUniformUnitNormalGenerator
from spykshrk.franklab.pp_decoder.util import normal_pdf_int_lookup, gaussian
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPEncoder, OfflinePPDecoder
from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer
from spykshrk.franklab.visualization import LinPosVisualizer, TetrodeVisualizer
from spykshrk.franklab.pp_decoder.util import apply_no_anim_boundary
from spykshrk.util import AttrDict



hv.extension('bokeh')
hv.extension('matplotlib')

hv.renderer('bokeh').theme = "dark_minimal"
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'

pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)



In [3]:
class InconsistentDataWarning(Warning):
    pass

In [4]:
# Encoding and decoding settings for both simulator and algorithm

@enum.unique
class Arm(enum.Enum):
    center = 1
    left = 2
    right = 3

@enum.unique
class Direction(enum.Enum):
    forward = 1
    reverse = 2
    
encode_settings = AttrDict({'sampling_rate': 1000,
                            'pos_bins': np.arange(0,800,1),
                            'pos_bin_edges': np.arange(0,800.1,1),
                            'pos_bin_delta': 1,
                            'pos_kernel': sp.stats.norm.pdf(np.arange(0,800,1), 400, 3),
                            'pos_kernel_std': 3, 
                            'mark_kernel_std': int(20), 
                            'pos_num_bins': 800,
                            'pos_col_names': [pos_col_format(ii, 800) for ii in range(800)],
                            'arm_coordinates': [[0, 100], [100, 250], [250,400], [400, 500], [500, 650], [650, 800]],
                            'wtrack_arm_coordinates': AttrDict(center=AttrDict(forward=AttrDict(x1=0, x2=100, len=100), 
                                                                               reverse=AttrDict(x1=400, x2=500, len=100)),
                                                               left=AttrDict(forward=AttrDict(x1=100, x2=250, len=150),
                                                                             reverse=AttrDict(x1=250, x2=400, len=150)),
                                                               right=AttrDict(forward=AttrDict(x1=500, x2=650, len=150),
                                                                              reverse=AttrDict(x1=650, x2=800, len=150))),
                            'vel': 3,
                            'spk_amp': 60})

decode_settings = AttrDict({'trans_smooth_std': 5,
                            'trans_uniform_gain': 0.001,
                            'time_bin_size': 10})
                            

In [5]:
def wtrack_simulate_arm_const_speed(arm_range, trial_time, trial_len, sampling_rate):
    pos_arm = np.arange(arm_range.x1, arm_range.x2, 
                        arm_range.len/sampling_rate/(trial_time*arm_range.len/(trial_len)))
    return pos_arm

def wtrack_simulate_trial_const_speed(arm1_range, arm1_enum, arm2_range, arm2_enum, trial_time, sampling_rate):
    pos_trial = np.array([])
    pos_trial_enum = np.array([])

    pos_arm = wtrack_simulate_arm_const_speed(arm1_range, trial_time, 
                                              arm1_range.len + arm2_range.len, sampling_rate)
    pos_trial = np.append(pos_trial, pos_arm)
    pos_trial_enum = np.append(pos_trial_enum, np.tile(arm1_enum, len(pos_arm)))

    pos_arm = wtrack_simulate_arm_const_speed(arm2_range, trial_time, 
                                              arm1_range.len + arm2_range.len, sampling_rate)
    pos_trial = np.append(pos_trial, pos_arm)
    pos_trial_enum = np.append(pos_trial_enum, np.tile(arm2_enum, len(pos_arm)))
    return pos_trial, pos_trial_enum

def wtrack_simulate_run_const_speed(arm1_dir, arm1_enum, arm2_dir, arm2_enum, trial_time, sampling_rate):
    pos_run = np.array([])
    pos_run_enum = np.array([])
    pos_run_dir_enum = np.array([])
    
    pos_trial, pos_trial_enum = wtrack_simulate_trial_const_speed(arm1_dir.forward, arm1_enum,
                                                                  arm2_dir.forward, arm2_enum, 
                                                                  trial_time, sampling_rate)
    pos_run = np.append(pos_run, pos_trial)
    pos_run_enum = np.append(pos_run_enum, pos_trial_enum)
    pos_run_dir_enum = np.append(pos_run_dir_enum, np.tile(Direction.forward, len(pos_trial)))

    pos_trial, pos_trial_enum = wtrack_simulate_trial_const_speed(arm2_dir.reverse, arm2_enum,
                                                                  arm1_dir.reverse, arm1_enum,
                                                                  trial_time, sampling_rate)
    pos_run = np.append(pos_run, pos_trial)
    pos_run_enum = np.append(pos_run_enum, pos_trial_enum)
    pos_run_dir_enum = np.append(pos_run_dir_enum, np.tile(Direction.reverse, len(pos_trial)))

    return pos_run, pos_run_enum, pos_run_dir_enum

def wtrack_simulate_series_const_speed(w_coor, trial_time, sampling_rate):
    pos_series = np.array([])
    pos_series_enum = np.array([])
    pos_series_dir_enum = np.array([])
    
    pos_run, pos_run_enum, pos_run_dir_enum = wtrack_simulate_run_const_speed(w_coor.center, Arm.center, 
                                                                              w_coor.left, Arm.left, 
                                                                              trial_time, sampling_rate)
    pos_series = np.append(pos_series, pos_run)
    pos_series_enum = np.append(pos_series_enum, pos_run_enum)
    pos_series_dir_enum = np.append(pos_series_dir_enum, pos_run_dir_enum)

    pos_run, pos_run_enum, pos_run_dir_enum = wtrack_simulate_run_const_speed(w_coor.center, Arm.center,
                                                                              w_coor.right, Arm.right,
                                                                              trial_time, sampling_rate)
    pos_series = np.append(pos_series, pos_run)
    pos_series_enum = np.append(pos_series_enum, pos_run_enum)
    pos_series_dir_enum = np.append(pos_series_dir_enum, pos_run_dir_enum)    
    
    return pos_series, pos_series_enum, pos_series_dir_enum
    

In [6]:
trial_time = 10

pos_epoch = np.array([])

pos_series, pos_series_enum, pos_series_dir_enum = wtrack_simulate_series_const_speed(encode_settings.wtrack_arm_coordinates, 
                                                                                      trial_time, encode_settings.sampling_rate)

pos_epoch = np.append(pos_epoch, pos_series)

pos_timestamp = np.arange(0, len(pos_epoch), 1)

pos_time = np.arange(0, pos_epoch.size/encode_settings.sampling_rate, 1/encode_settings.sampling_rate)

pos_vel = np.ones(len(pos_epoch)) * (pos_epoch[1] - pos_epoch[0]) / (pos_time[1] - pos_time[0])

# Duplicate
num_dup = 2
for ii in range(num_dup):
    pos_epoch = np.append(pos_epoch, pos_epoch)
    pos_timestamp = np.arange(0, len(pos_epoch), 1)
    pos_time = np.arange(0, pos_epoch.size/encode_settings.sampling_rate, 1/encode_settings.sampling_rate)
    pos_vel = np.append(pos_vel, pos_vel)
    pos_series_enum = np.append(pos_series_enum, pos_series_enum)
    pos_series_dir_enum = np.append(pos_series_dir_enum, pos_series_dir_enum)


In [7]:
linpos_flat = FlatLinearPosition.from_numpy_single_epoch(1, 1, pos_timestamp, pos_epoch, pos_vel, 
                                                         encode_settings.sampling_rate, 
                                                         encode_settings.wtrack_arm_coordinates)

linpos_flat['arm'] = pd.Series(index=linpos_flat.index, data=pos_series_enum, dtype='category')
linpos_flat['direction'] = pd.Series(index=linpos_flat.index, data=pos_series_dir_enum, dtype='category')

In [8]:
arm_colormap = dict(center='darkorange', left='pink', right='cyan')

direction_hatchmap = dict(forward='right_diagonal_line', reverse='left_diagonal_line')
mpl_direction_hatchmap = dict(forward='/', reverse='\\')

def wtrack_linear_plot_hook(plot, element, arm, direction):
    if hv.Store.current_backend == 'bokeh':
        plot.handles['glyph'].fill_color = arm_colormap[arm]
        plot.handles['glyph'].hatch_pattern = direction_hatchmap[direction]
        plot.handles['glyph'].line_color = None
        plot.handles['glyph'].hatch_color = 'grey'
    elif hv.Store.current_backend == 'matplotlib':
        element.opts(hatch=mpl_direction_hatchmap[direction], facecolor=arm_colormap[arm], clone=False)

def wtrack_linear_plot_init_hook(plot, element, arm, direction):
    if hv.Store.current_backend == 'bokeh':
        pass
    elif hv.Store.current_backend == 'matplotlib':
        plot_kwargs = plot.style.kwargs
        plot_kwargs['facecolor'] = color=arm_colormap[arm]
        plot.style = opts.Polygons(**plot_kwargs, hatch=mpl_direction_hatchmap[direction])

def pos_hook(plot, element):
    if hv.Store.current_backend == 'bokeh':
        pass
    elif hv.Store.current_backend == 'matplotlib':
        pass
    
def pos_init_hook(plot, element):
    if hv.Store.current_backend == 'bokeh':
        pass
    elif hv.Store.current_backend == 'matplotlib':
        pass
        
def wtrack_linear_plot_polygons(arm, direction, pos_time, w_coor):
    time_range = (pos_time[0], pos_time[-1])
    y_range = (w_coor[arm][direction].x1, w_coor[arm][direction].x2)
    time_total = time_range[1] - time_range[0]
    time_center = time_total/2 + time_range[0]
    y_total = y_range[1] - y_range[0]
    y_center = y_total/2 + y_range[0]
    
    box = hv.Box(time_center, y_center, (time_total, y_total))
    
    init_hooks = [functools.partial(wtrack_linear_plot_init_hook, arm=arm, direction=direction)]
    
    hooks = [functools.partial(wtrack_linear_plot_hook, arm=arm, direction=direction)]
    
    if hv.Store.current_backend == 'bokeh':
        poly = hv.Polygons(box).opts(hooks=hooks)
    elif hv.Store.current_backend == 'matplotlib':
        poly = hv.Polygons(box).opts(initial_hooks=init_hooks)
    return poly

def plot_position(time, pos, color='royalblue', fig_size=400, frame_width=800, aspect=3):
    if hv.Store.current_backend == 'bokeh':
        return hv.Scatter((time, pos)).opts(color=color, frame_width=frame_width, aspect=aspect)
    elif hv.Store.current_backend == 'matplotlib':
        return hv.Scatter((time, pos)).opts(color=color, fig_size=fig_size, aspect=aspect)
        


In [9]:
hv.output(backend='matplotlib')
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'
#hv.output(backend='matplotlib', fig='pdf')

w_coor = encode_settings.wtrack_arm_coordinates
wtrack_linear_plot_polygons(arm='center', direction='forward', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons(arm='left', direction='forward', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons( arm='left', direction='reverse', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons(arm='center', direction='reverse', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons(arm='right', direction='forward', pos_time=pos_time, w_coor=w_coor) * \
    wtrack_linear_plot_polygons(arm='right', direction='reverse', pos_time=pos_time, w_coor=w_coor) * \
    plot_position(pos_time, pos_epoch)
#hv.Scatter((pos_time, pos_epoch)).opts(color='royalblue', fig_size=400, aspect=2)

In [10]:
class WtrackLinposDecomposed(AttrDict):
    def __init__(self, encode_settings, bin_size=1):
        super().__init__()

        self.encode_settings = encode_settings
        self.wtrack_armcoord = self.encode_settings.wtrack_arm_coordinates
        
        self.bin_size = bin_size
        self.segment_order_cw = [('center', 'forward'),
                                 ('left', 'forward'), 
                                 ('left', 'reverse'),
                                 ('right', 'forward'),
                                 ('right', 'reverse'),
                                 ('center', 'reverse')]
        self.segment_order_ccw = [('center', 'forward'),
                                  ('right', 'forward'), 
                                  ('right', 'reverse'),
                                  ('left', 'forward'),
                                  ('left', 'reverse'),
                                  ('center', 'reverse')]
        self.armcoord_cw = self._segment_decomposed_with_buffer(self.segment_order_cw, self.wtrack_armcoord)
        self.armcoord_ccw = self._segment_decomposed_with_buffer(self.segment_order_ccw, self.wtrack_armcoord)
        self.armcoord_cw_num_bins = max([max((buffer.x1, buffer.x2)) for arm in self.armcoord_cw.values() 
                                         for direct in arm.values() for buffer in [direct.prev, direct.main, direct.next]])
        self.armcoord_ccw_num_bins = max([max((buffer.x1, buffer.x2)) for arm in self.armcoord_ccw.values() 
                                          for direct in arm.values() for buffer in [direct.prev, direct.main, direct.next]])
        self.wtrack_armcoord_cw = AttrDict({arm: AttrDict({direct: AttrDict(x1=min(direct_v.prev.x1, direct_v.main.x1, direct_v.next.x1),
                                                                            x2=max(direct_v.prev.x2, direct_v.main.x2, direct_v.next.x2))
                                                           for direct, direct_v in arm_v.items()})
                                            for arm, arm_v in self.armcoord_cw.items()})
        self.wtrack_armcoord_ccw = AttrDict({arm: AttrDict({direct: AttrDict(x1=min(direct_v.prev.x1, direct_v.main.x1, direct_v.next.x1),
                                                                             x2=max(direct_v.prev.x2, direct_v.main.x2, direct_v.next.x2))
                                                            for direct, direct_v in arm_v.items()})
                                             for arm, arm_v in self.armcoord_ccw.items()})
        self.cw_bin_range = [min([direct.x1 for arm in self.wtrack_armcoord_cw.values() for direct in arm.values()]),
                             max([direct.x2 for arm in self.wtrack_armcoord_cw.values() for direct in arm.values()])]
        self.cw_num_bins = self.cw_bin_range[1] - self.cw_bin_range[0]
        self.ccw_bin_range = [min([direct.x1 for arm in self.wtrack_armcoord_ccw.values() for direct in arm.values()]),
                              max([direct.x2 for arm in self.wtrack_armcoord_ccw.values() for direct in arm.values()])]
        self.ccw_num_bins = self.ccw_bin_range[1] - self.ccw_bin_range[0]
        self.wtrack_armcoord_main_cw = AttrDict({arm: AttrDict({direct: direct_v.main for direct, direct_v in arm_v.items()})
                                                 for arm, arm_v in self.armcoord_cw.items()})
        self.wtrack_armcoord_main_ccw = AttrDict({arm: AttrDict({direct: direct_v.main for direct, direct_v in arm_v.items()})
                                                  for arm, arm_v in self.armcoord_ccw.items()})
        self.simple_armcoord_cw, self.simple_armcoord_bins_cw = \
                self._create_wtrack_decomposed_simple_armcoord(self.armcoord_cw, self.bin_size)
        self.simple_armcoord_ccw, self.simple_armcoord_bins_ccw = \
                self._create_wtrack_decomposed_simple_armcoord(self.armcoord_ccw, self.bin_size)
        self.simple_main_armcoord_cw, self.simple_main_armcoord_bins_cw = \
                self._create_wtrack_decomposed_simple_main_armcoord(self.armcoord_cw, self.bin_size)
        self.simple_main_armcoord_ccw, self.simple_main_armcoord_bins_ccw = \
                self._create_wtrack_decomposed_simple_main_armcoord(self.armcoord_ccw, self.bin_size)
        self.wtrack_main_col_sel_ind, self.wtrack_main_col_sel, self.main_col_sel_ind, self.main_col_sel, \
                self.wtrack_prev_next_col_sel_ind, self.wtrack_prev_next_col_sel, \
                self.prev_next_col_sel_ind, self.prev_next_col_sel = \
                self._col_sel_main_prev_next(self.armcoord_cw, self.armcoord_ccw, self.wtrack_armcoord, 
                                        self.armcoord_cw_num_bins, self.encode_settings)
        
    @staticmethod
    def _segment_decomposed_with_buffer(segment_order, wtrack_arm_coord):
        seg_decomposed = AttrDict()
        seg_offset = 0
        for ii, seg in enumerate(segment_order):
            prev_seg = segment_order[ii-1]
            next_seg = segment_order[(ii+1)%len(segment_order)]

            prev_seg_len = wtrack_arm_coord[prev_seg[0]][prev_seg[1]].len
            main_seg_len = wtrack_arm_coord[seg[0]][seg[1]].len
            next_seg_len = wtrack_arm_coord[next_seg[0]][next_seg[1]].len

            prev_seg_start = seg_offset
            main_seg_start = prev_seg_start + prev_seg_len
            next_seg_start = main_seg_start + main_seg_len
            next_seg_end = next_seg_start + next_seg_len
            seg_total_len = prev_seg_len + main_seg_len + next_seg_len
            seg_offset += seg_total_len

            arm_dict = seg_decomposed.setdefault(seg[0], AttrDict())
            dir_dict = arm_dict.setdefault(seg[1], AttrDict(prev=AttrDict(x1=prev_seg_start, x2=main_seg_start, len=prev_seg_len), 
                                                            main=AttrDict(x1=main_seg_start, x2=next_seg_start, len=main_seg_len),
                                                            next=AttrDict(x1=next_seg_start, x2=next_seg_end, len=next_seg_len),
                                                            prev_seg=prev_seg,
                                                            next_seg=next_seg))
            
        return seg_decomposed
     
    @staticmethod
    def _create_wtrack_decomposed_simple_armcoord(decomp_armcoord, bin_size=1):
        simple_armcoord = [(min(direct.prev.x1, direct.main.x1, direct.next.x1),
                            max(direct.prev.x2, direct.main.x2, direct.next.x2))
                           for arm in decomp_armcoord.values() for direct in arm.values()]
        simple_armcoord.sort(key=lambda tup: tup[0])
        simple_armcoord_bins = np.array([])
        for seg in simple_armcoord:
            simple_armcoord_bins = np.append(simple_armcoord_bins, np.arange(seg[0], seg[1], bin_size))
        return simple_armcoord, simple_armcoord_bins

    @staticmethod
    def _create_wtrack_decomposed_simple_main_armcoord(decomp_armcoord, bin_size=1):
        simple_armcoord = [(direct.main.x1, direct.main.x2)
                           for arm in decomp_armcoord.values() for direct in arm.values()]
        simple_armcoord.sort(key=lambda tup: tup[0])
        simple_armcoord_bins = np.array([])
        for seg in simple_armcoord:
            simple_armcoord_bins = np.append(simple_armcoord_bins, np.arange(seg[0], seg[1], bin_size))
        return simple_armcoord, simple_armcoord_bins

    @staticmethod
    def _col_sel_main_prev_next(decomp_armcoord_cw, decomp_armcoord_ccw, wtrack_armcoord, decomp_num_bins, encode_settings):
        wtrack_map = [(arm, direct, direct_v.x1, direct_v.x2) 
                      for arm, arm_v in wtrack_armcoord.items() 
                      for direct, direct_v in arm_v.items()]
        main_col_sel = []
        wtrack_main_col_sel = []
        main_col_sel_ind = []
        wtrack_main_col_sel_ind = []
        prev_next_col_sel = {}
        wtrack_prev_next_col_sel = {}
        prev_next_col_sel_ind = {}
        wtrack_prev_next_col_sel_ind = {}
        for arm_range in wtrack_map:
            wtrack_main_seg = wtrack_armcoord[arm_range[0]][arm_range[1]]
            wtrack_main_seg_ind = list(range(wtrack_main_seg.x1, wtrack_main_seg.x2))
            wtrack_main_col_sel_ind.extend(wtrack_main_seg_ind)
            wtrack_main_col_sel.extend(pos_col_format(wtrack_main_seg_ind, encode_settings.pos_num_bins))
            
            main_seg = decomp_armcoord_cw[arm_range[0]][arm_range[1]].main
            main_col_seg_ind = list(range(main_seg.x1, main_seg.x2))
            main_col_sel_ind.extend(main_col_seg_ind)
            main_col_sel.extend(pos_col_format(main_col_seg_ind, decomp_num_bins))
            sel_rotate_keys = ['cw', 'ccw']
            sel_direct_keys = ['prev', 'next']
            for rotate_key in sel_rotate_keys:
                rotate_col_sel = prev_next_col_sel.setdefault(rotate_key, {})
                rotate_col_sel_ind = prev_next_col_sel_ind.setdefault(rotate_key, {})
                wtrack_rotate_col_sel = wtrack_prev_next_col_sel.setdefault(rotate_key, {})
                wtrack_rotate_col_sel_ind = wtrack_prev_next_col_sel_ind.setdefault(rotate_key, {})
                for direct_key in sel_direct_keys:
                    direct_col_sel = rotate_col_sel.setdefault(direct_key, [])
                    direct_col_sel_ind = rotate_col_sel_ind.setdefault(direct_key, [])
                    wtrack_direct_col_sel = wtrack_rotate_col_sel.setdefault(direct_key, [])
                    wtrack_direct_col_sel_ind = wtrack_rotate_col_sel_ind.setdefault(direct_key, [])
                    arm_sel = eval('decomp_armcoord_'+rotate_key)[arm_range[0]][arm_range[1]]
                    range_sel = arm_sel[direct_key]
                    wtrack_prev_next_arm, wtrack_prev_next_direct = arm_sel[direct_key+'_seg']
                    wtrack_prev_next_range = wtrack_armcoord[wtrack_prev_next_arm][wtrack_prev_next_direct]
                    wtrack_direct_col_sel_ind.extend(list(range(wtrack_prev_next_range.x1, wtrack_prev_next_range.x2)))
                    wtrack_direct_col_sel.extend(pos_col_format(list(range(wtrack_prev_next_range.x1, wtrack_prev_next_range.x2)), 
                                                                encode_settings.pos_num_bins))
                    
                    direct_col_sel_ind.extend(list(range(range_sel.x1, range_sel.x2)))
                    direct_col_sel.extend(pos_col_format(list(range(range_sel.x1, range_sel.x2)), decomp_num_bins))
        return wtrack_main_col_sel_ind, wtrack_main_col_sel, main_col_sel_ind, main_col_sel, wtrack_prev_next_col_sel_ind, \
               wtrack_prev_next_col_sel, prev_next_col_sel_ind, prev_next_col_sel

In [11]:
wtrack_decompose = WtrackLinposDecomposed(encode_settings)

In [12]:
encode_settings_decomp = AttrDict(encode_settings)
encode_settings_decomp.pos_num_bins = wtrack_decompose.armcoord_cw_num_bins
encode_settings_decomp.pos_col_names=[pos_col_format(ii, encode_settings_decomp.pos_num_bins) 
                                      for ii in range(encode_settings_decomp.pos_num_bins)]
encode_settings_decomp.pos_bins = np.arange(0, encode_settings_decomp.pos_num_bins)
encode_settings_decomp.pos_bin_edges = np.arange(0, encode_settings_decomp.pos_num_bins + 0.0001, 1)
encode_settings_decomp.pos_kernel_std = 3
encode_settings_decomp.pos_kernel = sp.stats.norm.pdf(np.arange(0, encode_settings_decomp.pos_num_bins, 1), 
                                                      encode_settings_decomp.pos_num_bins/2, 
                                                      encode_settings_decomp.pos_kernel_std)
encode_settings_decomp.pos_col_names = [pos_col_format(ii, encode_settings_decomp.pos_num_bins) 
                                        for ii in range(encode_settings_decomp.pos_num_bins)]
encode_settings_decomp.arm_coordinates = wtrack_decompose.simple_armcoord_cw
encode_settings_decomp.wtrack_decomp_arm_coordinates = wtrack_decompose.wtrack_armcoord_cw


In [13]:
def wtrack_remap_to_decomposed(linpos_flat, wtrack_arm_coord, wtrack_decomposed_cw, wtrack_decomposed_ccw):
    linpos_arm_dir = linpos_flat.groupby(['arm', 'direction'])

    decomposed_linpos_cw = pd.DataFrame()
    decomposed_linpos_ccw = pd.DataFrame()
    
    decomposed_linpos = pd.DataFrame()
    
    for entry in linpos_arm_dir:
        key = entry[0]
        table = entry[1]
        arm_coord_range = wtrack_arm_coord[key[0].name][key[1].name]
        decomposed_range_cw = wtrack_decomposed_cw[key[0].name][key[1].name]
        decomposed_range_ccw = wtrack_decomposed_ccw[key[0].name][key[1].name]
        decomposed_table = table.copy()
        decomposed_table.loc[:,'linpos_cw'] = decomposed_table.loc[:, 'linpos_flat'] - arm_coord_range.x1 + decomposed_range_cw.main.x1
        decomposed_table.loc[:,'linpos_ccw'] = decomposed_table.loc[:, 'linpos_flat'] - arm_coord_range.x1 + decomposed_range_ccw.main.x1
        
        decomposed_linpos = decomposed_linpos.append(decomposed_table)
    
    decomposed_linpos.sort_index(inplace=True)
    return decomposed_linpos

In [14]:
decomposed_linpos = wtrack_remap_to_decomposed(linpos_flat, encode_settings.wtrack_arm_coordinates,
                                               wtrack_decompose.armcoord_cw,
                                               wtrack_decompose.armcoord_ccw)

In [15]:
all_decon_range = np.array([(order.x1, order.x2) for arm in wtrack_decompose.armcoord_cw.values()
                            for direct in arm.values() for order in [direct.prev, direct.main, direct.next]])
decon_min = all_decon_range.min()
decon_max = all_decon_range.max()

In [16]:
hv.output(backend='bokeh', size=150)

hv.Points(decomposed_linpos['linpos_cw'], kdims=[('samples', 'Samples'), 
                                                 ('linpos_cw', 'Position (cm)')]).opts(aspect=2, ylim=(decon_min, decon_max))

In [17]:
pprint(wtrack_decompose.simple_main_armcoord_ccw,newline=' ',max_seq_length=100000)

In [18]:
tet_gen = TetrodeUniformUnitNormalGenerator(sampling_rate=encode_settings.sampling_rate,
                                            num_marks=4,
                                            num_units=100,
                                            mark_mean_range=(60, 200),
                                            mark_cov_range=(20, 40),
                                            firing_rate_range=(5, 40),
                                            pos_field_range=wtrack_decompose.simple_main_armcoord_cw,
                                            pos_field_bins=wtrack_decompose.simple_main_armcoord_bins_cw,
                                            pos_field_var_range=(2,10))

In [19]:
spk_amps, unit_spks = tet_gen.simulate_tetrode_over_pos(decomposed_linpos, col_name='linpos_cw')

In [20]:
tet_viz = TetrodeVisualizer(spk_amps, decomposed_linpos, unit_spks)
tet_viz.plot_color_3d_dynamic('linpos_cw', 'c00', 'c01')

In [80]:
%%time
#%%prun -r -s cumulative

# Setup encoding model and estimate the position distribution of each spike being encoded
encoder_cw = OfflinePPEncoder(linflat=decomposed_linpos, enc_spk_amp=spk_amps, dec_spk_amp=spk_amps, 
                              encode_settings=encode_settings_decomp, decode_settings=decode_settings,
                              linflat_col_name='linpos_cw')
encoder_ccw = OfflinePPEncoder(linflat=decomposed_linpos, enc_spk_amp=spk_amps, dec_spk_amp=spk_amps, 
                               encode_settings=encode_settings_decomp, decode_settings=decode_settings,
                               linflat_col_name='linpos_ccw')
observ_cw = encoder_cw.run_encoder()
observ_ccw = encoder_ccw.run_encoder()


In [22]:
hv.output(backend='matplotlib')

# Setup plot to visualize estimated position distribution
sel_distrib = observ_cw.loc[:, pos_col_format(0,encode_settings_decomp.pos_num_bins):         
                            pos_col_format(encode_settings_decomp.pos_num_bins-1,
                                           encode_settings_decomp.pos_num_bins)]
    
sel_pos = observ_cw.loc[:, 'position']
max_prob = sel_distrib.max().max()
    
def plot_observ(ind):
    plot_list = []
    for ii in range(5):
        plot_list.append(hv.Curve(sel_distrib.iloc[ind+ii], extents=(0, 0, 2050, max_prob)))
        plot_list.append(hv.Points((sel_pos.iloc[ind+ii], [0.005])))
    return hv.Overlay(plot_list).opts(fig_size=400, aspect=3)
        
dmap = hv.DynamicMap(plot_observ, kdims=['ind'])
dmap.redim.values(ind=list(range(0, len(observ_cw)-5, 5)))

In [81]:
def decomposed_remap_to_wtrack(pos, decomposed_armcoord, wtrack_armcoord):
    wtrack_pos = np.empty(pos.shape)
    wtrack_pos.fill(np.nan)
    for arm_k, arm_v in decomposed_armcoord.items():
        for direct_k, direct_v in arm_v.items():
            dir_pos_ind = (pos >= direct_v.main.x1) & (pos <= direct_v.main.x2)
            wtrack_dir_pos = pos[dir_pos_ind] - direct_v.main.x1 + wtrack_armcoord[arm_k][direct_k].x1
            wtrack_pos[dir_pos_ind] = wtrack_dir_pos
    
    if np.isnan(wtrack_pos).any():
        warnings.warn('Position in decomposed main coordinate system does not match wtrack coordinates.', InconsistentDataWarning)
        
    return wtrack_pos

In [82]:
def wtrack_recompose_observ_to_wtrack(observ_cw, prob_no_spike_cw, trans_mat_cw,
                                      observ_ccw, prob_no_spike_ccw, trans_mat_ccw, wtrack_decompose, encode_settings):
    decomp_armcoord_cw = wtrack_decompose.armcoord_cw
    decomp_armcoord_ccw = wtrack_decompose.armcoord_ccw
    wtrack_armcoord = wtrack_decompose.wtrack_armcoord
    wtrack_map = [(arm, direct, direct_v.x1, direct_v.x2) 
                  for arm, arm_v in wtrack_armcoord.items() 
                  for direct, direct_v in arm_v.items()]
    prev_next_list = [list(prev_next_seg)
                      for arm, arm_v in decomp_armcoord_cw.items() 
                      for direct, direct_v in arm_v.items() 
                      for prev_next_seg in [itertools.chain(direct_v.prev_seg, (direct_v.prev.x1, direct_v.prev.x2)),
                                            itertools.chain(direct_v.next_seg, (direct_v.next.x1, direct_v.next.x2))]]
    prev_next_map = AttrDict()
    prev_next_grp = itertools.groupby(prev_next_list, key=lambda x: (x[0], x[1]))
    for p_n_key, p_n_iter in prev_next_grp:
        arm = prev_next_map.setdefault(p_n_key[0], AttrDict())
        direct = arm.setdefault(p_n_key[1], [])
        for p_n_val in p_n_iter:
            direct.append(p_n_val[2:])
            
    observ = observ_cw.loc[:, wtrack_decompose.main_col_sel].iloc[:, wtrack_decompose.wtrack_main_col_sel_ind].copy()
    observ.columns = encode_settings.pos_col_names
    prob_no_spike = {}
    for tet_id, prob_no_spike_tet in prob_no_spike_cw.items():
        prob_no_spike[tet_id] = prob_no_spike_tet[wtrack_decompose.main_col_sel_ind]
    
    #trans_main_ax_mask = np.zeros(trans_mat_cw.shape[0])
    #trans_main_ax_mask[wtrack_decompose.main_col_sel_ind] = 1
    #trans_mat = trans_mat_cw[trans_main_ax_mask[:, np.newaxis] & trans_main_ax_mask].reshape((encode_settings.pos_num_bins,
    #                                                                                          encode_settings.pos_num_bins))
    trans_mat = np.zeros([encode_settings.pos_num_bins]*2)
    trans_main_sel = np.meshgrid(wtrack_decompose.main_col_sel_ind, wtrack_decompose.main_col_sel_ind)
    trans_wtrack_sel = np.meshgrid(wtrack_decompose.wtrack_main_col_sel_ind, wtrack_decompose.wtrack_main_col_sel_ind)
    trans_mat[trans_wtrack_sel[1], trans_wtrack_sel[0]] = trans_mat_cw[trans_main_sel[1], trans_main_sel[0]]

    for rotate_key, rotate_col_sel in wtrack_decompose.prev_next_col_sel.items():
        for direct_key, direct_col_sel in rotate_col_sel.items():
            wtrack_prev_next_col_sel_ind = wtrack_decompose.wtrack_prev_next_col_sel_ind[rotate_key][direct_key]
            observ = (observ + eval('observ_'+rotate_key).loc[:, direct_col_sel].
                      iloc[:, wtrack_prev_next_col_sel_ind].values)
            dir_col_sel_ind = wtrack_decompose.prev_next_col_sel_ind[rotate_key][direct_key]
            trans_dir_sel = np.meshgrid(dir_col_sel_ind, dir_col_sel_ind)
            trans_wtrack_prev_next_sel = np.meshgrid(wtrack_prev_next_col_sel_ind, wtrack_prev_next_col_sel_ind)
            trans_mat[trans_wtrack_prev_next_sel[1], trans_wtrack_prev_next_sel[0]] += \
                    eval('trans_mat_'+rotate_key)[trans_dir_sel[1], trans_dir_sel[0]]
            for tet_id in prob_no_spike.keys():
                prob_no_spike[tet_id] += eval('prob_no_spike_'+rotate_key)[tet_id][dir_col_sel_ind]
    
    observ = observ.join(observ_cw.get_other_view())
    observ = SpikeObservation.create_default(observ, enc_settings=wtrack_decompose.encode_settings)

    observ['position'] = decomposed_remap_to_wtrack(observ['position'], wtrack_decompose.armcoord_cw, wtrack_decompose.wtrack_armcoord)
    
    return observ, prob_no_spike, trans_mat


In [88]:
mat = encoder_cw.trans_mat['learned']
hv.output(backend='bokeh')
trans_mat = np.zeros((encode_settings.pos_num_bins, encode_settings.pos_num_bins))
main_sel = np.meshgrid(wtrack_decompose.main_col_sel_ind, wtrack_decompose.main_col_sel_ind)
wtrack_sel = np.meshgrid(wtrack_decompose.wtrack_main_col_sel_ind, wtrack_decompose.wtrack_main_col_sel_ind)
trans_mat[wtrack_sel[1], wtrack_sel[0]] = mat[main_sel[1], main_sel[0]]
img = hv.Image(np.flip(trans_mat, axis=0), bounds=[0,0,800,800]) 
img = img.opts(hv.opts.Image(invert_yaxis=True))
img2 = hv.Image(np.flip(mat[main_sel[1], main_sel[0]], axis=0), bounds=[0,0,800,800]) 
img2 = img2.opts(hv.opts.Image(invert_yaxis=True))
img + img2

In [94]:
mesh = np.meshgrid(wtrack_decompose.prev_next_col_sel_ind['cw']['prev'], wtrack_decompose.prev_next_col_sel_ind['cw']['prev'])
hv.Image(encoder_cw.trans_mat['learned'][mesh[1], mesh[0]])


In [84]:
observ, prob_no_spike, trans_mat = wtrack_recompose_observ_to_wtrack(observ_cw, encoder_cw.prob_no_spike, encoder_cw.trans_mat['learned'],
                                                                     observ_ccw, encoder_ccw.prob_no_spike, encoder_ccw.trans_mat['learned'],
                                                                     wtrack_decompose, encode_settings)


In [102]:
np.matmul(np.array(range(25)).reshape([5,5]), np.array([1,1,0,0,0]))

In [87]:
hv.Image(encoder_cw.trans_mat['learned'])

In [86]:
hv.Image(trans_mat)

In [ ]:
decomposed_remap_to_wtrack(observ['position'], wtrack_decompose.armcoord_cw, wtrack_decompose.wtrack_armcoord)

In [ ]:
hv.output(backend='matplotlib')

# Setup plot to visualize estimated position distribution
sel_distrib = observ.loc[:, pos_col_format(0,encode_settings.pos_num_bins):         
                         pos_col_format(encode_settings.pos_num_bins-1,
                                        encode_settings.pos_num_bins)]
    
sel_pos = observ.loc[:, 'position']
max_prob = sel_distrib.max().max()

def plot_observ(ind):
    plot_list = []
    for ii in range(5):
        plot_list.append(hv.Curve(sel_distrib.iloc[ind+ii], extents=(0, 0, 800, max_prob)))
        plot_list.append(hv.Points((sel_pos.iloc[ind+ii], [0.005])))
    return hv.Overlay(plot_list).opts(fig_size=400, aspect=3)
        
dmap = hv.DynamicMap(plot_observ, kdims=['ind'])
dmap.redim.values(ind=list(range(0, len(sel_distrib), 5)))

In [ ]:
%%time
# Run PP decoding algorithm
time_bin_size = 10

decoder = OfflinePPDecoder(observ_obj=observ, trans_mat=encoder.trans_mat['learned'], 
                           prob_no_spike=prob_no_spike,
                           encode_settings=encode_settings, decode_settings=decode_settings, 
                           time_bin_size=time_bin_size)

posteriors = decoder.run_decoder()

In [ ]:
display(wtrack_decompose.armcoord_ccw)
display(wtrack_decompose.armcoord_cw)

In [ ]:
wtrack_decompose.wtrack_armcoord

In [ ]:
wtrack_decompose.wtrack_armcoord

In [ ]:
wtrack_decompose.armcoord_ccw

In [ ]:
test = pd.DataFrame(np.ones((10,10)), columns=list(range(10)))
test.columns.isin([1,4])

In [ ]:
num_bins = 1000
format_col = lambda x: 'x{:0{dig}d}'.format(x, dig=len(str(num_bins)))

In [ ]:
df = pd.DataFrame(np.zeros((2,2)))
df.columns = [3,4]
df + np.array([[1,2], [3,4]])